참고 : [[코드] 데이콘 기초 베이스라인](https://dacon.io/competitions/official/235670/codeshare/1738?page=1&dtype=recent)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터 살펴보기

In [2]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re
import os

In [3]:
# 경로 설정
os.chdir('/content/drive/MyDrive/Dacon/소설_작가_분류_AI_경진대회/data/')

In [4]:
# # 압축 풀기
# !unzip -qq "/content/drive/MyDrive/Dacon/소설_작가_분류_AI_경진대회/data/235670_data.zip"

In [5]:
print(os.getcwd())

/content/drive/MyDrive/Dacon/소설_작가_분류_AI_경진대회/data


In [6]:
#파일 불러오기
train = pd.read_csv('train.csv', encoding = 'utf-8')
test = pd.read_csv('test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('sample_submission.csv', encoding = 'utf-8')

In [7]:
#train 데이터 살펴보기
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [8]:
#test 데이터 살펴보기
test

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


In [9]:
#sample_submission
sample_submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
...,...,...,...,...,...,...
19612,19612,0,0,0,0,0
19613,19613,0,0,0,0,0
19614,19614,0,0,0,0,0
19615,19615,0,0,0,0,0


# 전처리

In [10]:
#부호를 제거해주는 함수
def alpha_num(text):
  return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [11]:
#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [12]:
# 오류 발생 : AttributeError: 'str' object has no attribute 'splite'
# #불용어 제거해주는 함수
# def remove_stopwords(text):
#     final_text = []
#     for i in text.splite():
#       if i.strip().lower() not in stopwords:
#         final_text.append(i.strip())
#     return "".join(final_text)

# # 불용어
# stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
#              "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
#              "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
#              "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
#              "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
#              "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
#              "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
#              "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
#              "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
#              "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
#              "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [13]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [14]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [15]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [16]:
X_train

array(['almost choking much much wanted say strange exclamations came lips pole gazed fixedly bundle notes hand looked odin evident perplexity',
       'sister asked suppose',
       'engaged one day walked perusing janes last letter dwelling passages proved jane not written spirits instead surprised mr odin saw looking odin meeting putting away letter immediately forcing smile said',
       ..., 'sincere wellwisher friend sister lucy odin',
       'wanted lend money', 'certainly not occurred said yes like'],
      dtype='<U1433')

In [17]:
y_train

array([3, 2, 1, ..., 1, 3, 0])

# 모델링
<span style='background-color: #F7DDBE'>22.03.24</span>

In [18]:
# 파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [19]:
#tokenizer에 fit
# 가장 빈도가 높은 20,000개의 단어만 선택하도록하는 Tokenizer 객체
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
# 그에 맞게 단어 인덱스를 구축
tokenizer.fit_on_texts(X_train)

# 계산된 단어 인덱스를 dictionary 형태로 확인
word_index = tokenizer.word_index

In [20]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
# 문자열을 인덱스의 리스트로 변환
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padding = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [22]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [24]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 16)           320000    
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense (Dense)               (None, 24)                408       
                                                                 
 dense_1 (Dense)             (None, 5)                 125       
                                                                 
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


In [25]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train,
                    epochs=num_epochs, verbose=2,
                    validation_split=0.2)

Epoch 1/20
1372/1372 - 11s - loss: 1.5635 - accuracy: 0.2788 - val_loss: 1.5379 - val_accuracy: 0.2722 - 11s/epoch - 8ms/step
Epoch 2/20
1372/1372 - 6s - loss: 1.4130 - accuracy: 0.4235 - val_loss: 1.2799 - val_accuracy: 0.4794 - 6s/epoch - 4ms/step
Epoch 3/20
1372/1372 - 4s - loss: 1.1559 - accuracy: 0.5512 - val_loss: 1.0912 - val_accuracy: 0.5879 - 4s/epoch - 3ms/step
Epoch 4/20
1372/1372 - 5s - loss: 0.9987 - accuracy: 0.6145 - val_loss: 0.9827 - val_accuracy: 0.6317 - 5s/epoch - 3ms/step
Epoch 5/20
1372/1372 - 5s - loss: 0.8919 - accuracy: 0.6616 - val_loss: 0.9094 - val_accuracy: 0.6575 - 5s/epoch - 4ms/step
Epoch 6/20
1372/1372 - 5s - loss: 0.8163 - accuracy: 0.6941 - val_loss: 0.8925 - val_accuracy: 0.6533 - 5s/epoch - 4ms/step
Epoch 7/20
1372/1372 - 5s - loss: 0.7588 - accuracy: 0.7189 - val_loss: 0.8292 - val_accuracy: 0.6887 - 5s/epoch - 4ms/step
Epoch 8/20
1372/1372 - 4s - loss: 0.7139 - accuracy: 0.7353 - val_loss: 0.8155 - val_accuracy: 0.6950 - 4s/epoch - 3ms/step
Epoch 

In [29]:
# # predict values
# pred = model.predict_proba(test_padding)

AttributeError: ignored

In [34]:
# # 오류 해결1
# predict_prob=model.predict([testa,testb])

# predict_classes=np.argmax(predict_prob,axis=1)

In [37]:
# 오류 해결2
y_pred = model.predict(test_padding)
print(y_pred.shape)
y_pred

(19617, 5)


array([[5.5953446e-05, 3.3884516e-01, 5.7043429e-02, 6.0328126e-01,
        7.7419373e-04],
       [2.5608924e-01, 5.0667864e-01, 5.0128810e-02, 3.9861131e-02,
        1.4724214e-01],
       [9.9993396e-01, 5.4228258e-06, 6.7254899e-09, 1.4348929e-08,
        6.0626266e-05],
       ...,
       [2.2382343e-04, 9.9976021e-01, 7.8422380e-09, 1.4275364e-05,
        1.6872294e-06],
       [9.3168659e-05, 9.9988008e-01, 1.0306288e-07, 1.6641099e-05,
        9.9826448e-06],
       [9.9963260e-01, 2.4001224e-07, 7.7900393e-07, 1.8522034e-07,
        3.6624225e-04]], dtype=float32)

In [39]:
# submission
sample_submission[['0','1','2','3','4']] = y_pred
sample_submission

,index,0,1,2,3,4
0,0,5.595345e-05,3.388452e-01,5.704343e-02,6.032813e-01,7.741937e-04
1,1,2.560892e-01,5.066786e-01,5.012881e-02,3.986113e-02,1.472421e-01
2,2,9.999340e-01,5.422826e-06,6.725490e-09,1.434893e-08,6.062627e-05
3,3,3.399359e-05,6.423776e-09,9.803615e-01,2.666599e-06,1.960178e-02
4,4,9.618499e-01,2.862415e-04,5.611180e-05,3.775161e-02,5.622702e-05
...,...,...,...,...,...,...
19612,19612,1.465584e-07,9.999999e-01,5.664568e-21,4.631953e-10,1.163708e-15
19613,19613,2.416026e-02,1.610482e-05,4.126075e-04,1.567335e-09,9.754111e-01
19614,19614,2.238234e-04,9.997602e-01,7.842238e-09,1.427536e-05,1.687229e-06
19615,19615,9.316866e-05,9.998801e-01,1.030629e-07,1.664110e-05,9.982645e-06


In [40]:
sample_submission.to_csv('submission.csv', index = False, encoding = 'utf-8')


1. tf.keras.preprocessing.sequence.pad_sequences
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
2. tf.keras.preprocessing.text.Tokenizer
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
3. 파이썬 경로/디렉토리 위치
https://velog.io/@stu_dy/Python-%ED%98%84%EC%9E%AC-%ED%8C%8C%EC%9D%BC%EB%94%94%EB%A0%89%ED%86%A0%EB%A6%AC-%EC%9C%84%EC%B9%98-%ED%99%95%EC%9D%B8-%EB%B0%8F-%EB%B3%80%EA%B2%BD
4. apply() : https://wikidocs.net/46758
5. strip() : https://codechacha.com/ko/python-string-strip/
6. np.array() : https://datascienceschool.net/01%20python/03.01%20%EB%84%98%ED%8C%8C%EC%9D%B4%20%EB%B0%B0%EC%97%B4.html
7. tokenizer : https://inuplace.tistory.com/536
8. tf.keras.Sequential : https://keras.io/ko/getting-started/sequential-model-guide/
9. tf.keras.layers.Embedding : https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding
10. model.compile : https://blog.naver.com/PostView.nhn?blogId=shwotjd14&logNo=221458224334&categoryNo=0&parentCategoryNo=0&viewDate=&currentPage=1&postListTopCurrentPage=1&from=postView
11. 오류 해결2 : https://stackoverflow.com/questions/67460062/neural-network-and-binary-classification-guidance/67467084#67467084